## Import Statements

In [ ]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy
import os
import pickle
    
from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
# modelBeingUsed = "glove-wiki-gigaword-300"
# modelBeingUsed = "glove-wiki-gigaword-100"
# modelBeingUsed = "glove-twitter-50"
modelBeingUsed = "glove-twitter-100"
# modelBeingUsed = "glove-twitter-200"

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words, dataClean


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Downloading binaries and models


In [ ]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower() or "n" in tstString.lower()):
    print(" didnt reload model! ")
else:
    print("loading {0}!".format(modelBeingUsed));
    fileName = "{}.pickle".format(modelBeingUsed)
    if(os.path.exists(pathToDatasets+fileName)):
        print("loading via pickle!")
        pickle_in = open(pathToDatasets+fileName, "rb")
        word_vectors = pickle.load(pickle_in);
    else:
        print("Pickle didn't exist, therefore loading model!")
        word_vectors = api.load("{0}".format(modelBeingUsed))
        print("-- Saving to pickle file for next time! --")
        pickle_out = open(pathToDatasets+fileName,"wb")
        pickle.dump(word_vectors, pickle_out)
        pickle_out.close()
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    print("Model Loaded!")

## Global Variables and Global Objects

In [ ]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 5
TWEET_START = 1
NUM_OF_TWEETS = 2


# VERBOSE_PRINTING = True
VERBOSE_PRINTING = False

# USE_SPACY = False
USE_SPACY = True

COLOR_PRINTING = True
#COLOR_PRINTING = False

PRINT_NEUTRAL = True
# PRINT_NEUTRAL = False

PRINT_ALL_STRINGS = True
# PRINT_ALL_STRINGS = False

SHOW_ALTS = 50

punctuation = r"\"#$%&'+-/;<=>?@[\]*^_`{|}~"
LIST_OF_NEGATIONS = ["not"]

## Class for Sentences


In [ ]:
from SentenceClass import *

## Utility Code

In [ ]:
def printStrings(sentenceObj):
    
    numberOfPrints = 0
    newStrings = generateHTMLObjectsFromSentence(sentenceObj)
    mainSentiment = sentenceObj.ogSentiment;
    listOfSentencesWithSentiments = []
    bestSentiment = -sys.maxsize - 1
    worstSentiment = sys.maxsize
    
    bestSentimentString = "";
    worstSentimentString = "";
    
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            newObj = SentenceWithSentiment(alteredTweet, sentimentOfNewString, htmlText)
            listOfSentencesWithSentiments.append(newObj)
            
            if(sentimentOfNewString >= bestSentiment):
                bestSentiment = sentimentOfNewString
                bestSentimentString = htmlText;
            elif(sentimentOfNewString < worstSentiment):
                worstSentiment = sentimentOfNewString
                worstSentimentString = htmlText;
                
            
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'black')
                numberOfPrints+=1;
            elif(sentimentOfNewString > mainSentiment):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'green')
                numberOfPrints+=1;
            elif(sentimentOfNewString < mainSentiment and sentimentOfNewString != 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'red')
                numberOfPrints+=1;
                
    if(numberOfPrints > SHOW_ALTS): print("--- More options (total: {0}) possible, but not printed ---".format(numberOfPrints));
    if (worstSentimentString != ""): displayText("Worst Sentence: {0} : {1}".format(worstSentimentString, worstSentiment), color='red')
    if (bestSentimentString != ""): displayText("Best Sentence: {0} : {1}".format(bestSentimentString, bestSentiment), color='green') 
    
    
    sentenceObj.addFinalSentences(listOfSentencesWithSentiments)
    return listOfSentencesWithSentiments, sentenceObj;

def cstr(s, color='black', italics=False):
    if(COLOR_PRINTING):
        if(italics):
            return cstr("<i>{0}</i>".format(s), color);
        return "<text style=color:{}>{}</text>".format(color, s)
    else:
        return "{}".format(s)

def displayText(text, color='black'):
    if(COLOR_PRINTING):
        display(HTML(cstr(text, color)));
        return
    print("{}".format(text));
    
    
def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.tag_) for token in doc] # since the format expected is [text,tag]
    return tags

def getAntonymsAndSynonymsOfWords(word):
    if(word not in vocabulary):
        return [], []
    setOfAntonyms = set()
    setOfSynonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            setOfSynonyms.add(l.name().lower())
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name().lower())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    if(len(setOfSynonyms) == 0):
        if(VERBOSE_PRINTING): print("No synonyms found for word {0}".format(word))
            
    return list(setOfAntonyms), list(setOfSynonyms)


def returnReplacementsForWord(word):
    
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms,synonyms = getAntonymsAndSynonymsOfWords(word)
    
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
    if(synonyms != []):
        possibleReplacements.extend(synonyms)
        if(VERBOSE_PRINTING): print("Some synonyms for word {0} are {1}".format(word, synonyms[:3]))
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        if("_" in word): continue
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        if(str(newTag) == str(mainTag)):
            if(VERBOSE_PRINTING): print("Word {0}[{1}] matched with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
    

def removeNegationsFromSentence(sentenceObj):
    finalSentenceObjects = sentenceObj.getFinalSentences();
    newObjs = []
    for obj in finalSentenceObjects:
        thisSentence = cleanAndTokenizeText(obj.getSentence());
        if("not" in thisSentence):
            notIndex = thisSentence.index("not")
            if(notIndex >= len(thisSentence) - 1): continue
            if(senty.polarity_scores(thisSentence[notIndex+1])['compound'] != 0):
                newSentence = thisSentence[:notIndex] + thisSentence[notIndex+1:] # indexing works as [)
                newSentence = ' '.join(newSentence)
                newSentiment = senty.polarity_scores(newSentence)['compound']
                newObjs.append(SentenceWithSentiment(newSentence, newSentiment))
                sentenceObj.addAlternativesByIndex(notIndex,["IGNORE_FLAG"])
           
    sentenceObj.addFinalSentences(newObjs)
    return sentenceObj


def generateHTMLObjectsFromSentence(sentenceObj):
    
    allSentences = sentenceObj.getFinalSentences()
    indexToAlts = sentenceObj.indexToSetOfWords;
    indexToChange = list(indexToAlts.keys());
    
    listOfSentenceObjs = []
    for t_sentenceObj in allSentences:
        sentence = t_sentenceObj.getSentence()
        copySentence = cleanAndTokenizeText(sentence)
        for index in indexToChange:
            if(index >= len(copySentence)): continue
            if("IGNORE_FLAG" in indexToAlts[index] and copySentence[index] in LIST_OF_NEGATIONS): continue
            copySentence[index] = cstr("[{0}]".format(copySentence[index]), "blue", italics=True);
        listOfSentenceObjs.append(SentenceWithHTML(sentence, ' '.join(copySentence)));
    
    return listOfSentenceObjs

    
    

## Sentence Chunking and Appending

In [ ]:
def helper_combine(mainList, myList):
    '''
    helper function for CombineSentenceChunks
    '''
    newList = []
    for val in myList:
        for mainVal in mainList:
#             if(VERBOSE_PRINTING): print("Combining {0} with {1}".format(' '.join(val), ' '.join(mainVal)));
            newList.append(val + mainVal);
    return newList;

def combineSentenceChunks(wholeSentence, dictOfChunks):
    '''
        Uses the helper_combine function to generate all possible combinatios and permuations of the chunks
        and any alternatives.
        
        The logic is to use the end of the sentence, apply each possible chunk from the previous key's chunks
        to every possible chunk of this key's.
        
        The helper function is used to allow us to reuse the list of alreadyGeneratedChunks and constantly
        append to them.
        
        To understand the logic better, take a look at this gist:
        https://gist.github.com/sunnyMiglani/cf85407a9e6928237b1436cc2bc95fa4
        
    '''
    reversedKeys = sorted(dictOfChunks.keys(), reverse=True)
    completeSentences = [];
    mainArr = dictOfChunks[reversedKeys[0]]
    for ind in range(1, len(reversedKeys)):
        key = reversedKeys[ind];
        mainArr = helper_combine(mainArr, dictOfChunks[key]);
        
    return mainArr;
        
def generateSentenceChunks(wholeSentence, keyToChange, nextKey, listOfMyAlternatives):
    '''
        Generates sentence chunks by iterating through the list of alternatives
        Chunking the sentence to start from current key to next key.
        This means that the sentence always goes from key 'x' to key 'y'
        
        Example:
        "I really <hate> hot chocolate, but I <prefer> hot coffee"
        Calling generateSentenceChunks will create an example sentence:
            - "<altWordForHate> hot chocolate , but I "
        
        Remember to append the first stretch of the string to the first key's chunk for proper use!
    '''
    
    newList = list(listOfMyAlternatives)
    newList.append(wholeSentence[keyToChange]);
    generatedSentences = []
    for myAlt in newList:
        newSentence = wholeSentence[:]
        newSentence[keyToChange] = myAlt
#         if(VERBOSE_PRINTING): print("Generated : {}".format(newSentence[keyToChange:nextKey]))
        generatedSentences.append(newSentence[keyToChange:nextKey]);
        
    return generatedSentences
    
def returnCombinationsOfStrings(sentenceObj):
    
    indexToWordDict = sentenceObj.indexToSetOfWords;
    originalSentence = sentenceObj.ogSentence;
    tokenizedSentence = cleanAndTokenizeText(originalSentence)
    reversedKeys = sorted(indexToWordDict.keys(), reverse=True)
    dictAlternatives  = {}

    sortedKeys = sorted(indexToWordDict.keys())
    sentenceChunks = {}
    htmlChunks = {}
    
    for ind in range(0,len(sortedKeys)):
        key = sortedKeys[ind]
        nextKey = sortedKeys[ind+1] if ind+1 < len(sortedKeys) else len(tokenizedSentence)
        sentenceChunks[key] = generateSentenceChunks(tokenizedSentence, key, nextKey, indexToWordDict[key])

    if(sortedKeys[0] != 0):
        newList = []
        for thislist in sentenceChunks[sortedKeys[0]]:
            newList.append(tokenizedSentence[0:sortedKeys[0]] + thislist)
        sentenceChunks[sortedKeys[0]] = newList;
        
    finalOptions = combineSentenceChunks(tokenizedSentence, sentenceChunks)
    
    finalSentences = []
    for val in finalOptions:
        sentence = ' '.join(val)
        sentiment = senty.polarity_scores(sentence)['compound']
        finalSentences.append(SentenceWithSentiment(sentence,sentiment))
    
    sentenceObj.addFinalSentences(finalSentences)   
    return sentenceObj



        

In [ ]:
def getAlternativeWords(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;

    sentenceTokens = cleanAndTokenizeText(mainSentence)

    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        copyOfTokens = sentenceTokens[:]
        replacements = []
        
        score = senty.polarity_scores(word)['compound']
        if(score != 0.0):
            replacements = sentenceObj.checkIfWordExists(word)
            if(replacements != []):
                print("FOUND REPEATED WORD {0}".format(word))
                continue;
            tempReplacements = returnReplacementsForWord(word) # get embedding based relations
            if(VERBOSE_PRINTING): print("Some early replacements for word [{0}] are {1}".format(word, tempReplacements[0:8]))
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            finalReplacements = []
            if(VERBOSE_PRINTING): skippedWords = []
            for new_word in replacements:
                thisSentiment = senty.polarity_scores(new_word)['compound']
                if(thisSentiment != 0.0):
                    finalReplacements.append(new_word)
                else:
                    if(VERBOSE_PRINTING):
#                         print("Sentiment of Skipped word {} is {}".format(word, senty.polarity_scores(word)))
                        skippedWords.append(new_word)
            if(VERBOSE_PRINTING and len(skippedWords) > 0):
                print("some skipped words: {0}".format(skippedWords[:5]));
            if(finalReplacements == []):
                if(VERBOSE_PRINTING): print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, finalReplacements)
            sentenceObj.addWordToAlternatives(word, finalReplacements)
    return sentenceObj




In [ ]:
def interactionHelper(dictOfAltObjs, sentiments, thisSent, lastWorkingIndex=0):
    if(thisSent in sentiments):
        mainIndex = sentiments.index(thisSent)
        
    else:
        mainIndex = lastWorkingIndex
    
    numLower =  mainIndex  # Mod to keep it in range 0 -> val 
    numHigher = len(sentiments) - mainIndex - 1

    print("There are totally {0} possible sentences".format(len(sentiments)))
    print("Current sentiment is at index {0}, and length of sentiments is {1}".format(mainIndex, len(sentiments)));
    print(" <--- {0} lower and {1} ---> higher ".format(numLower, numHigher))
    print("Would you like to see a lower sentiment or higher sentiment or ALL?")
    print("Please enter 1 for lower, 2 for higher or 0 for all")
    
    targetSentiment = thisSent
    textInput = input()
    if(textInput == ""):
        print("You entered nothing! exiting")
        return
        
    if(textInput.isdigit()):
        textInput = int(textInput)
        if(textInput == 0):
            if(VERBOSE_PRINTING): print("Printing sentiments from : {0}".format(sentiments))
            for sentiment in sentiments:
                obj = dictOfAltObjs[sentiment][0];
                displayText("{0}  : {1}".format(obj.getHTML(), sentiment))
            return
        elif(textInput == 1 and numLower > 0 and (mainIndex - 1) >= 0):
            targetSentiment = sentiments[mainIndex - 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        elif(textInput == 2 and numHigher < len(sentiments) and (mainIndex + 1) < len(sentiments)):
            targetSentiment = sentiments[mainIndex + 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        else:
            print("You entered an invalid option")
            interactionHelper(dictOfAltObjs, sentiments, thisSent, mainIndex)
            return
        
    interactionHelper(dictOfAltObjs, sentiments, targetSentiment,mainIndex)



def interactionWithUser(mainSentenceObj, dictOfAltObjs,sentiments, isUser=False):
    if(isUser == False):
        return
    else:
        mainSentence = mainSentenceObj.ogSentence;
        mainSent = mainSentenceObj.ogSentiment;
        if(VERBOSE_PRINTING):
            print("The following are the sentiment values for the input")
            for ind,sent in enumerate(sentiments):
                print("{0} - {1}".format(ind+1,sent))
        
        interactionHelper(dictOfAltObjs, sentiments, mainSent) 

In [ ]:
def extractTwitterDataset():
    df_tweets = pd.read_csv( pathToDatasets + 'cleanedTweets.csv', nrows=NUM_OF_TWEETS, skiprows =TWEET_START)
    tweets = df_tweets.values
    return tweets;

def createDictionaryOfSentStrings(sentencesWithSentiment):
    dict_sentimentToStringObjects = {}
    for obj in sentencesWithSentiment:
        sent = obj.getSentiment()
        if(dict_sentimentToStringObjects.get(sent) == None):
            dict_sentimentToStringObjects[sent] = [obj]
        else:
            (dict_sentimentToStringObjects[sent]).append(obj)

    return dict_sentimentToStringObjects;

def runThroughTweets():
    tweets = extractTwitterDataset()
    
    counterOfTweets = 0;
    for counter,tweet in enumerate(tweets):
        counterOfTweets +=1;
        tweet = tweet[0]
        tweetTokens = cleanAndTokenizeText(tweet)
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n\n Tweet: {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeWords(sentenceObj)
        replacementDictionary = sentenceObj.getDictOfIndexWords();
        if(len(replacementDictionary) <= 0):        
            print(" -- No new Strings generated ---\n\n")
            continue

        keysToChange = replacementDictionary.keys();
        for key in keysToChange:
            if(VERBOSE_PRINTING):print("{0}'th word's ({2}) options: {1}".format(key,replacementDictionary[key],tweetTokens[key]))

        sentenceObj = returnCombinationsOfStrings(sentenceObj)
        sentenceObj = removeNegationsFromSentence(sentenceObj)
        sentencesWithSentiment, sentenceObj = printStrings(sentenceObj)
       
        dict_sentimentToStringObjects = createDictionaryOfSentStrings(sentencesWithSentiment)
        
        sentiments = sorted(dict_sentimentToStringObjects.keys())
        
    return counterOfTweets;
        
numOfTweets = runThroughTweets()
print("num of tweets done : {0}".format(numOfTweets))

## Testing Code


In [ ]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

In [ ]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    textString = dataClean(textString)
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeWords(sentenceObj)
    replacementDictionary = sentenceObj.getDictOfIndexWords();
    if(len(replacementDictionary) <= 0):        
        print(" -- No new Strings generated ---\n\n")
        return
    
    keysToChange = replacementDictionary.keys();
    if(VERBOSE_PRINTING):
        for key in keysToChange:
            print("{0}'th word's options: {1}".format(key,replacementDictionary[key]))
        
    
    sentenceObj = returnCombinationsOfStrings(sentenceObj)
    sentenceObj = removeNegationsFromSentence(sentenceObj)
    allPossibleSentences, sentenceObj = printStrings(sentenceObj)
    
    
    dict_sentimentToStringObjects = createDictionaryOfSentStrings(allPossibleSentences)
        
    sentiments = sorted(dict_sentimentToStringObjects.keys())
#     interactionWithUser(sentenceObj, dict_sentimentToStringObjects, sentiments[0:10], False);

print("Enter a string to use")


inputText = input()
if(inputText == ""):
    return
elif(inputText == "t"):
    specificString("I really love hot chocolate, but I'm not good with hot coffee ")
else:
    specificString(inputText)
# I really love hot chocolate, but I'm not good with hot coffee 


this is making me feel bad:-0.5423
this is making me feel a bit bad:-0.5423

this is making me feel very bad:-0.5849
this is making me feel quite bad:-0.5849
this is making me feel incredibly bad:-0.5849

this is making me feel kinda bad:-0.4951
this is making me feel slightly bad:-0.4951
this is making me feel barely bad:-0.4951

this is making me feel not bad:0.431
